In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",  # Changed from Phi-2 to Mistral 7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.3: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2024.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Preparation for Mental Health Chatbot

In [4]:
mental_health_prompt = """Below is a conversation between a mental health assistant and a user. The assistant is empathetic, supportive, and provides helpful information.

User: {}

Assistant: {}"""

def format_prompt(sample):
    return {
        "prompt": mental_health_prompt.format(sample["user_input"], ""),
        "response": sample["assistant_response"]
    }

# Note: In a real scenario, you would use a proper mental health dataset.
# For demonstration purposes, we're using a small example dataset.
from datasets import Dataset

mental_health_dataset = [
    {"user_input": "I've been feeling really anxious lately. What can I do?",
     "assistant_response": "I'm sorry to hear you're feeling anxious. It's a common experience, but that doesn't make it any less challenging. Some strategies that might help include deep breathing exercises, regular physical activity, and mindfulness practices. It's also important to ensure you're getting enough sleep and maintaining a balanced diet. If your anxiety persists or becomes overwhelming, consider reaching out to a mental health professional for additional support."},
    {"user_input": "I'm having trouble sleeping. Any tips?",
     "assistant_response": "Difficulty sleeping can be frustrating. Here are some tips that might help: establish a consistent sleep schedule, create a relaxing bedtime routine, avoid screens for at least an hour before bed, ensure your sleeping environment is dark and cool, and avoid caffeine and heavy meals close to bedtime. If sleep problems persist, it may be helpful to consult with a healthcare provider to rule out any underlying issues."}
]

dataset = Dataset.from_list(mental_health_dataset)
dataset = dataset.map(format_prompt)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

<a name="Train"></a>
### Training the Model

In [6]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="prompt",
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=5e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        output_dir="mental_health_model",
        report_to="none",  # Disable wandb logging
    ),
    packing=False,  # Disable packing here
)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


<a name="Inference"></a>
### Inference

In [7]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    mental_health_prompt.format(
        "I'm feeling overwhelmed with work and personal life. How can I manage stress better?",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

<s> Below is a conversation between a mental health assistant and a user. The assistant is empathetic, supportive, and provides helpful information.

User: I'm feeling overwhelmed with work and personal life. How can I manage stress better?

Assistant:  I understand that you're feeling overwhelmed. Stress is a normal part of life, but it's important to manage it effectively. Here are some tips to help you manage stress:

1. Take breaks: Take regular breaks throughout the day to relax and recharge. This can help you feel more refreshed and less stressed.
2. Exercise: Regular exercise can help reduce stress and improve your overall well-being.
3. Practice mindfulness: Mindfulness is a technique that involves focusing on the present moment and letting go of worries about the past or future. This can help you feel more calm and relaxed.
4. Talk to someone: Talking to a friend, family member, or mental health professional can help you feel supported and less alone.
5. Seek professional help

<a name="Save"></a>
### Saving the Model

In [8]:
model.save_pretrained("mental_health_model")
tokenizer.save_pretrained("mental_health_model")

('mental_health_model/tokenizer_config.json',
 'mental_health_model/special_tokens_map.json',
 'mental_health_model/tokenizer.model',
 'mental_health_model/added_tokens.json',
 'mental_health_model/tokenizer.json')